# E11. 인공지능 작사가 만들기

# 11-7. 프로젝트: 멋진 작사가 만들기

## 1) 데이터 다운로드

## 2) 데이터 읽어오기

In [1]:
# tf
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해

import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/e/e11_lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Look, I was gonna go easy on you and not to hurt your feelings', "But I'm only going to get this one chance", "Something's wrong, I can feel it (Six minutes, Slim Shady, you're on)"]


## Step 3. 데이터 정제
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

---
`preprocess_sentence()` 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가면 잘라내기를 권합니다.

In [3]:
print(type(raw_corpus))
print(len(raw_corpus))
raw_corpus[:10]

<class 'list'>
187088


['Look, I was gonna go easy on you and not to hurt your feelings',
 "But I'm only going to get this one chance",
 "Something's wrong, I can feel it (Six minutes, Slim Shady, you're on)",
 "Just a feeling I've got, like something's about to happen, but I don't know what",
 "If that means, what I think it means, we're in trouble, big trouble,",
 "And if he is as bananas as you say, I'm not taking any chances",
 "You were just what the doctor ordered I'm beginning to feel like a Rap God, Rap God",
 'All my people from the front to the back nod, back nod',
 'Now who thinks their arms are long enough to slap box, slap box?',
 'They said I rap like a robot, so call me Rapbot But for me to rap like a computer must be in my genes']

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence


maxlen 설정 시, n단어를 넘어가는 문장의 나머지 부분은 쓰지 않고 버려버리기에 오히려 문법상, 문맥상 의미가 없어진다고 판단하여 `len(list.split())`이 15개 이상, 즉 한 문장의 길이가 긴 경우 고려하지 않도록 설정했다.    
이런 식으로 설정하니 추후 param 갯수가 천만 대를 초과하지 않았으며, 학습 또한 비교적 빠른 시간 내에 실행할 수 있었다.

In [5]:
# 길이가 0인 문장 정제
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence.split()) >= 15: continue
    
    corpus.append(preprocess_sentence(sentence))


len(corpus)
corpus[:3]

['<start> look , i was gonna go easy on you and not to hurt your feelings <end>',
 '<start> but i m only going to get this one chance <end>',
 '<start> something s wrong , i can feel it six minutes , slim shady , you re on <end>']

In [6]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=10000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 133   4 ...   0   0   0]
 [  2  32   5 ...   0   0   0]
 [  2 195  16 ...   0   0   0]
 ...
 [  2   5  61 ...   0   0   0]
 [  2 113 657 ...   0   0   0]
 [  2   8  50 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f0b3c68a990>


## Step 4. 평가 데이터셋 분리
훈련 데이터와 평가 데이터를 분리하세요!

---

`tokenize()` 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 `train_test_split()` 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!

```python
enc_train, enc_val, dec_train, dec_val = <코드 작성>
```
여기까지 올바르게 진행했을 경우, 아래 실행 결과를 확인할 수 있습니다.
```python
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
```
```python
out:

Source Train: (124960, 14)
Target Train: (124960, 14)
```

만약 결과가 다르다면 천천히 과정을 다시 살펴 동일한 결과를 얻도록 하세요!

In [7]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0], src_input.shape)
print(tgt_input[0], src_input.shape)

[   2  133    4    5   57   99   54  418   18    7    8   68   10  424
   21 1026    3    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0] (166325, 32)
[ 133    4    5   57   99   54  418   18    7    8   68   10  424   21
 1026    3    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0] (166325, 32)


In [8]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input,
                                                          random_state=2020, test_size=0.2)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input)

VOCAB_SIZE = tokenizer.num_words + 1

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [10]:
print(dataset, "\n", val_dataset)

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)> 
 <BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>


## Step 5. 인공지능 만들기

---

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 `val_loss` 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!


```python
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
```

Q4. 모델이 생성한 가사 한 줄을 제출하세요.

In [22]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.fnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.fnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.fnn_1(out)
        out = self.fnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words+1, embedding_size, hidden_size)

In [23]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 32, 10001), dtype=float32, numpy=
array([[[-4.53679968e-05,  3.25786823e-04,  1.75364563e-04, ...,
          4.05189785e-05, -1.90848732e-04, -2.00633236e-04],
        [-3.87281762e-05,  7.77760812e-04,  2.90703116e-04, ...,
          4.27653140e-04, -3.42163519e-04, -4.52699169e-04],
        [ 1.07326341e-05,  1.15496211e-03,  7.62828160e-04, ...,
          1.04057114e-03, -3.13268480e-04, -5.07660618e-04],
        ...,
        [-2.64499034e-03,  1.95209740e-03,  8.91469463e-05, ...,
          1.53649447e-03, -7.72142038e-03,  4.21432592e-03],
        [-2.67033256e-03,  1.99263287e-03,  7.12552428e-05, ...,
          1.51124236e-03, -7.73690408e-03,  4.23988653e-03],
        [-2.69060652e-03,  2.02699122e-03,  5.43223432e-05, ...,
          1.49045046e-03, -7.74812698e-03,  4.26130928e-03]],

       [[-4.53679968e-05,  3.25786823e-04,  1.75364563e-04, ...,
          4.05189785e-05, -1.90848732e-04, -2.00633236e-04],
        [-4.45816142e-04,  4.84901946e-04, -2

In [24]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  5120512   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  6295552   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  10251025  
Total params: 30,059,793
Trainable params: 30,059,793
Non-trainable params: 0
_________________________________________________________________


In [25]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction="none")

model.compile(loss = loss, optimizer=optimizer, metrics=['accuracy'])

model.fit(dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
519/519 [==============================] - 157s 302ms/step - loss: 1.6523 - accuracy: 0.7596 - val_loss: 1.4226 - val_accuracy: 0.7751
Epoch 2/10
519/519 [==============================] - 151s 291ms/step - loss: 1.3662 - accuracy: 0.7791 - val_loss: 1.3320 - val_accuracy: 0.7819
Epoch 3/10
519/519 [==============================] - 151s 290ms/step - loss: 1.2883 - accuracy: 0.7844 - val_loss: 1.2789 - val_accuracy: 0.7860
Epoch 4/10
519/519 [==============================] - 187s 360ms/step - loss: 1.2283 - accuracy: 0.7888 - val_loss: 1.2388 - val_accuracy: 0.7894
Epoch 5/10
519/519 [==============================] - 186s 359ms/step - loss: 1.1768 - accuracy: 0.7928 - val_loss: 1.2093 - val_accuracy: 0.7922
Epoch 6/10
519/519 [==============================] - 154s 296ms/step - loss: 1.1302 - accuracy: 0.7968 - val_loss: 1.1830 - val_accuracy: 0.7955
Epoch 7/10
519/519 [==============================] - 161s 311ms/step - loss: 1.0869 - accuracy: 0.8009 - val_loss: 1.1609 -

In [30]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [31]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i want you <end> '

In [32]:
generate_text(model, tokenizer, init_sentence="<start> i", max_len=20)

'<start> i m the one , yeah , yeah <end> '

In [33]:
generate_text(model, tokenizer, init_sentence="<start> i will", max_len=20)

'<start> i will eat you alive <end> '

In [43]:
generate_text(model, tokenizer, init_sentence="<start> here i", max_len=20)

'<start> here i am , baby , i m a star <end> '

In [36]:
generate_text(model, tokenizer, init_sentence="<start> why", max_len=20)

'<start> why you wanna be with me <end> '

In [46]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love is a beautiful thing <end> '

다양한 명사들을 넣어 text generation을 해 본 결과 모든 문장들이 그럴듯하게 생성되었다.    
다른 분들이 maxlen설정의 유/무에 따라 파라미터 갯수, 학습시간 등이 양분되었음을 감안할 때 속도가 빠르면서도 문장 중간에 maxlen번째 단어까지만 취급하여 반쪽짜리 문장을 넣지 않는 모델을 설계하여 좋은 결과를 얻을 수 있었다.
```
love is a beautiful thing!
```